In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import Literal

In [ ]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["서울"]:
        return "기온은 섭씨 15도이고 흐립니다."
    else:
        return "기온은 섭씨 30도이고 화창합니다."

In [ ]:
get_weather.invoke(input={"location": "서울"})

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_kwargs =  { 
        "temperature": 0,
        "top_p": 1,
}

tools = [get_weather]

llm = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
).bind_tools(tools)

In [ ]:
llm.invoke("안녕?")

In [ ]:
llm.invoke("서울의 날씨는 어때?")

In [ ]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [ ]:
workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

In [ ]:
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
from langchain_core.messages import HumanMessage

messages1 = [HumanMessage(content="안녕?")]
messages2 = [HumanMessage(content="서울의 날씨는 어때?")]

In [ ]:
graph.invoke({"messages": messages1})

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="그 도시는 어떤게 유명해?")
        ]
    }
)

### Add Memory to chatbot

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
graph.invoke(
    {"messages": [HumanMessage(content="서울의 날씨는 어때?")]},
    config={"configurable": {"thread_id": 1}},
)

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="그 도시는 어떤게 유명해?")
        ]
    },
    config={"configurable": {"thread_id": 1}},
)